# Importando os dados

In [1]:
import requests,json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas.util.testing as pd
import numpy as np

C:\Users\andres\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  after removing the cwd from sys.path.


### Transformada do Cosseno

In [2]:
import math, time
def dct(p):
    n = len(p)
    mat = []
    suma = 0
    cte=math.sqrt(2/n)
    cf = 0
    for i in range(n):
        if i==0: cf = 1/math.sqrt(2)
        else: cf = 1
        for t in range(n):
            theta = ((2*t+1)*i*math.pi)/(2*n)
            suma = suma + p[t]*math.cos(theta)
        suma = cf*suma*cte
        mat.append(np.round(suma, decimals=3))
        suma = 0
    return mat

def idct(g):
    n = len(g)
    cte = math.sqrt(2/n)
    imat = []
    suma = 0
    cj = 0
    for i in range(n):
        suma = 0
        for t in range(n):
            if t==0: cj=1/math.sqrt(2)
            else: cj=1
            theta = ((2*i+1)*t*math.pi)/(2*n)
            suma = suma + cj*g[t]*math.cos(theta)
        suma = suma*cte
        imat.append(np.round(suma, decimals=3))
    return imat

## Lidando com os dados

In [4]:
#Função para importar os dados
def import_data(ano_i: int,ano_f: int, lat: list):
    dic = []
    F107 = []
    RSN = []
    factor = 0
    dias = 365
    tec_lat = []
    
    for lat_i in lat:
        for ano in range(ano_i, ano_f+1):
            parameters_tec = {"ano": ano, "lat": lat_i,"dia_i": 1,"dia_f": dias}
            response = requests.get("https://tecdatas.herokuapp.com/tec", params=parameters_tec)
            dic = dic + response.json()
        tec_lat.append(dic)
        dic = []
        
    for ano in range(ano_i,ano_f+1):
        parameters_datas = {"ano": ano, "dia_i": 1,"dia_f": dias}
        response = requests.get("https://tecdatas.herokuapp.com/f107", params=parameters_datas)
        F107 = F107 + response.json()
        parameters_datas = {"ano": ano, "dia_i": 1,"dia_f": dias}
        response = requests.get("https://tecdatas.herokuapp.com/rsn", params=parameters_datas)
        RSN = RSN + response.json()
        factor += 1
    return tec_lat,F107,RSN

# Função para filtrar os dados
def filtra_dados(ano_i: int, ano_f: int, lon: int, lat:float, dic: list, F107: list, RSN: list):
    dados = []
    dias = 365
    f107 = []
    rsn = []
    step = 0
    factor = ano_f-ano_i + 1
    
    #Selecionando os dados para uma dada longitude
    for i in range(13*dias*factor):
            dados.append(dic[lat][i][str(lon)])#Longitude -50.0

    #Criando e colocando o F10.7 e RSN em uma lista
    for ano in range(ano_i,ano_f+1):
        for f in range(step, step+365):
            f107.append(F107[f][str(ano)])
        step=step+365
        
    step = 0
    for ano in range(ano_i,ano_f+1):
        for f in range(step, step+365):
            rsn.append(RSN[f][str(ano)])
        step=step+365
    
    return dados,f107,rsn

# Função para salvar os dados em um dataFrame
def salva_dataframe(ano_i:int, ano_f:int, dados: list, f107: list, rsn: list):
    index = ['coef '+str(i) for i in range(1,14,1)] + ['F10.7'] + ['Sunspot'] 
    TEC_f = pd.DataFrame(index=index) #Tec na frequência
    step = 0
    dias = 365
    factor = ano_f-ano_i + 1
    for dias in range(1,factor*dias+1):
        data = dados[step:step+13] + [f107[dias-1]] + [rsn[dias-1]] 
        data = np.array(data).reshape(15,1)
        TEC_f[dias] = data
        step = step+13


    #Vamos aplicar aqui a DCT ao DataFrame TEC_f
    for i in range(1,dias+1):
        TEC_f[i][0:13] = dct(list(TEC_f[i][0:13]))  

    TEC_f = TEC_f.transpose()
    return TEC_f

def correlacao(TEC_f):
    corr_1 = [] #F10.7
    corr_2 = [] #Sunspot
    for i in range(1,14):
        filt = pd.DataFrame(TEC_f['coef '+str(i)])
        filt['F10.7'] = TEC_f['F10.7']
        filt['Sunspot'] = TEC_f['Sunspot']
        a = filt.corr()
        corr_1 = corr_1 + [a['coef '+str(i)][1]]
        corr_2 = corr_2 + [a['coef '+str(i)][2]]
    return corr_1, corr_2

In [6]:
lat = []
step = 50
while step != -52.5:
    lat.append(step)
    step -= 2.5

longitude = -50
ano_i = 2016
ano_f = 2019

tec_lat, F107, RSN = import_data(ano_i,ano_f,lat)

KeyboardInterrupt: 

In [ ]:
longitude = -50
corr_sunspot = []
corr_f107 = []
for lat_i in lat:
    #Filtrando os dados para determinada latitude
    dados, f107, sunspot = filtra_dados(ano_i,ano_f,longitude, lat_i, tec_lat, F107, RSN)
    
    #Salvando em um dataframe os dados filtrados
    TEC_f = salva_dataframe(ano_i,ano_f, dados, f107, sunspot)
    
    #Calculo da correlação
    a, b = correlacao(TEC_f)
    corr_f107.append(a)
    corr_sunspot.append(b)
    

## Realizando a análise

In [ ]:
images_dir = '/resultados_latitudevariante/'

index = ['coef '+str(i) for i in range(1,14)]
correlacao = pd.DataFrame(index=index)
for i in range(len(lat)):
    correlacao[str(lat[i])] = np.array(corr_sunspot[i]).reshape(13,1)

lat_str = str(lat).split(',')
lat_str[0] = lat_str[0][1::]
lat_str[len(lat)-1] = lat_str[len(lat)-1][::-1]
lat_str[len(lat)-1] = lat_str[len(lat)-1][1::]
lat_str[len(lat)-1] = lat_str[len(lat)-1][::-1]


fig, ax = plt.subplots(figsize=(20,10))
ax.set_xticks(lat)
ax.set_xticklabels(lat_str, fontsize=15)
ax.set_xlabel('Latitudes')
ax.set_ylabel('Coeficientes da DCT')
sns.set(font_scale=2)
graph = sns.heatmap(abs(correlacao), ax=ax)
ax.set_title('Heatmap da correlação Sunspot', fontdict={'fontsize':24})
fig.savefig(f"{images_dir}/sunspot.png")